### Find valid genes to perturb, i.e. genes within the top x genes in the rank-value encoding of any cell embedding

In [2]:
import sys
import os
import pickle
from tqdm import tqdm

sys.path.append('/work/magroup/kaileyhu/Geneformer')

from datasets import load_from_disk, concatenate_datasets

from geneformer import perturber_utils as pu
TOKEN_DICTIONARY_FILE = "/work/magroup/kaileyhu/Geneformer/geneformer/token_dictionary_gc95M.pkl"

In [10]:
# set name of dataset to get genes from
hvg = False # full datset or only top 500 hvgs?

if hvg:
    dataset_name = "/work/magroup/kaileyhu/res/hvg_500_tokenized.dataset"
else:
    dataset_name = "/work/magroup/kaileyhu/res/all_genes_tokenized.dataset"

In [11]:
ds = load_from_disk(dataset_name)

In [12]:
with open(TOKEN_DICTIONARY_FILE, "rb") as td:
    gene_token_dict = pickle.load(td)

token_gene_dict = {v: k for k, v in gene_token_dict.items()}

In [13]:
special_tokens = ["<pad>", "<cls>", "<eos>"]

In [15]:
gene_list = []

for patient in tqdm(ds['input_ids']):
    for gene in patient:
        # print(f"patient length is {len(patient)}")
        ensembl_id = token_gene_dict[gene]
        if ensembl_id not in special_tokens and ensembl_id not in gene_list:
            gene_list.append(ensembl_id)

100%|██████████| 1479/1479 [04:16<00:00,  5.76it/s]


In [16]:
if hvg:
    with open("/work/magroup/kaileyhu/res/top_2048_hvg_NEW.pkl", 'wb') as hvg_file:
        pickle.dump(gene_list, hvg_file)

else:
    with open("/work/magroup/kaileyhu/res/top_4096_NEW.pkl", 'wb') as all_file:
        pickle.dump(gene_list, all_file)